# CNC Data Exploration

This notebook explores the CNC manufacturing data from the CIMCO MDC Software database to understand patterns, data quality, and relationships that will inform our machine learning models.

## Objectives
1. Connect to the MySQL database and load data
2. Perform initial data quality assessment
3. Explore data distributions and patterns
4. Identify key relationships for modeling
5. Generate insights for feature engineering

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import warnings

# Import project modules
import sys
sys.path.append('../src')

from data.database import DatabaseManager
from data.preprocessing import DataPreprocessor
from utils.helpers import validate_data_quality, setup_logging

# Configuration
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Set up logging
logger = setup_logging()

## 1. Data Loading and Initial Inspection

In [ ]:
# Initialize database connection
print("Connecting to database...")
db = DatabaseManager()

# Test connection
connection_status = db.test_connection()
print(f"Database connection: {'Success' if connection_status else 'Failed'}")

if connection_status:
    # Get table information
    table_info = db.get_table_info()
    print(f"\nTable: {table_info['table_name']}")
    print(f"Columns: {table_info['column_count']}")
    
    # Display column information
    print("\nColumn Information:")
    for col, info in table_info['columns'].items():
        print(f"  {col}: {info['type']} {'(nullable)' if info['nullable'] else '(required)'}")

In [ ]:
# Load data (start with a sample for exploration)
print("Loading data sample...")
df_sample = db.get_all_data(limit=5000)  # Start with 5k records

print(f"Loaded {len(df_sample)} records")
print(f"Date range: {df_sample['StartTime'].min()} to {df_sample['StartTime'].max()}")

# Display basic info
df_sample.info()

In [ ]:
# Display first few rows
print("First 5 rows:")
df_sample.head()

## 2. Data Quality Assessment

In [ ]:
# Perform comprehensive data quality validation
quality_report = validate_data_quality(df_sample)

print("DATA QUALITY REPORT")
print("="*50)
print(f"Total records: {quality_report['total_records']:,}")
print(f"Duplicate records: {quality_report['duplicate_records']:,}")

print("\nMissing Values:")
for col, missing in quality_report['missing_values'].items():
    if missing > 0:
        pct = (missing / quality_report['total_records']) * 100
        print(f"  {col}: {missing:,} ({pct:.1f}%)")

print("\nNegative Duration Values:")
for col, negative in quality_report['negative_durations'].items():
    pct = (negative / quality_report['total_records']) * 100
    print(f"  {col}: {negative:,} ({pct:.1f}%)")

print(f"\nInvalid dates (1969): {quality_report['invalid_dates']:,}")

print("\nOutliers:")
for col, outliers in quality_report['outliers'].items():
    pct = (outliers / quality_report['total_records']) * 100
    print(f"  {col}: {outliers:,} ({pct:.1f}%)")

In [ ]:
# Missing values visualization
missing_data = df_sample.isnull().sum()
missing_data = missing_data[missing_data > 0].sort_values(ascending=False)

if len(missing_data) > 0:
    plt.figure(figsize=(12, 6))
    missing_data.plot(kind='bar')
    plt.title('Missing Values by Column')
    plt.xlabel('Columns')
    plt.ylabel('Missing Count')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
else:
    print("No missing values found in the sample data.")

## 3. Basic Statistics and Distributions

In [ ]:
# Summary statistics for numerical columns
print("NUMERICAL COLUMNS SUMMARY")
print("="*50)
numerical_cols = df_sample.select_dtypes(include=[np.number]).columns
df_sample[numerical_cols].describe()

In [ ]:
# Categorical columns summary
print("CATEGORICAL COLUMNS SUMMARY")
print("="*50)

categorical_cols = ['machine', 'State', 'OperatorName', 'PartNumber']

for col in categorical_cols:
    if col in df_sample.columns:
        unique_count = df_sample[col].nunique()
        print(f"\n{col}:")
        print(f"  Unique values: {unique_count}")
        
        if unique_count < 20:  # Show all if less than 20
            value_counts = df_sample[col].value_counts()
            print("  Distribution:")
            for value, count in value_counts.head(10).items():
                print(f"    {value}: {count}")
        else:
            print("  Top 5 most common:")
            value_counts = df_sample[col].value_counts()
            for value, count in value_counts.head(5).items():
                print(f"    {value}: {count}")

In [ ]:
# Calculate efficiency if not present
if 'efficiency' not in df_sample.columns:
    df_sample['efficiency'] = np.where(
        df_sample['JobDuration'] > 0,
        df_sample['RunningTime'] / df_sample['JobDuration'],
        0
    )
    # Cap efficiency at 100%
    df_sample['efficiency'] = df_sample['efficiency'].clip(0, 1)

# Calculate total downtime
downtime_cols = ['SetupTime', 'WaitingSetupTime', 'NotFeedingTime', 'AdjustmentTime',
                'DressingTime', 'ToolingTime', 'EngineeringTime', 'MaintenanceTime',
                'BuyInTime', 'BreakShiftChangeTime', 'IdleTime']

available_downtime_cols = [col for col in downtime_cols if col in df_sample.columns]
df_sample['total_downtime'] = df_sample[available_downtime_cols].sum(axis=1)

# Calculate parts per hour if possible
if 'PartsProduced' in df_sample.columns:
    df_sample['parts_per_hour'] = np.where(
        df_sample['RunningTime'] > 0,
        df_sample['PartsProduced'] * 3600 / df_sample['RunningTime'],
        0
    )
    # Handle infinite values
    df_sample['parts_per_hour'] = df_sample['parts_per_hour'].replace([np.inf, -np.inf], 0)

print("Calculated derived metrics: efficiency, total_downtime, parts_per_hour")

## 4. Key Performance Metrics Exploration

In [ ]:
# Efficiency distribution
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Efficiency histogram
axes[0,0].hist(df_sample['efficiency'], bins=50, alpha=0.7, color='skyblue', edgecolor='black')
axes[0,0].set_title('Efficiency Distribution')
axes[0,0].set_xlabel('Efficiency')
axes[0,0].set_ylabel('Frequency')
axes[0,0].axvline(df_sample['efficiency'].mean(), color='red', linestyle='--', label=f'Mean: {df_sample["efficiency"].mean():.3f}')
axes[0,0].legend()

# Job duration distribution (log scale)
axes[0,1].hist(np.log1p(df_sample['JobDuration']), bins=50, alpha=0.7, color='lightgreen', edgecolor='black')
axes[0,1].set_title('Job Duration Distribution (log scale)')
axes[0,1].set_xlabel('Log(Job Duration + 1)')
axes[0,1].set_ylabel('Frequency')

# Total downtime distribution
axes[1,0].hist(df_sample['total_downtime'] / 3600, bins=50, alpha=0.7, color='salmon', edgecolor='black')
axes[1,0].set_title('Total Downtime Distribution (Hours)')
axes[1,0].set_xlabel('Total Downtime (Hours)')
axes[1,0].set_ylabel('Frequency')

# Parts per hour distribution (if available)
if 'parts_per_hour' in df_sample.columns:
    # Filter out extreme outliers for visualization
    parts_filtered = df_sample[df_sample['parts_per_hour'] <= df_sample['parts_per_hour'].quantile(0.95)]
    axes[1,1].hist(parts_filtered['parts_per_hour'], bins=50, alpha=0.7, color='gold', edgecolor='black')
    axes[1,1].set_title('Parts per Hour Distribution')
    axes[1,1].set_xlabel('Parts per Hour')
    axes[1,1].set_ylabel('Frequency')
else:
    axes[1,1].text(0.5, 0.5, 'Parts per Hour\nNot Available', ha='center', va='center', transform=axes[1,1].transAxes)

plt.tight_layout()
plt.show()

## 5. Machine Performance Analysis

In [ ]:
# Machine performance comparison
machine_stats = df_sample.groupby('machine').agg({
    'efficiency': ['mean', 'std', 'count'],
    'JobDuration': 'mean',
    'total_downtime': 'mean'
}).round(3)

# Flatten column names
machine_stats.columns = ['_'.join(col).strip() for col in machine_stats.columns]
machine_stats = machine_stats.rename(columns={
    'efficiency_mean': 'avg_efficiency',
    'efficiency_std': 'efficiency_std',
    'efficiency_count': 'job_count',
    'JobDuration_mean': 'avg_job_duration',
    'total_downtime_mean': 'avg_total_downtime'
})

# Sort by efficiency
machine_stats = machine_stats.sort_values('avg_efficiency', ascending=True)

print("MACHINE PERFORMANCE SUMMARY")
print("="*60)
print(machine_stats)

# Visualize machine performance
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Average efficiency by machine
machine_stats['avg_efficiency'].plot(kind='barh', ax=axes[0], color='steelblue')
axes[0].set_title('Average Efficiency by Machine')
axes[0].set_xlabel('Efficiency')

# Job count by machine
machine_stats['job_count'].plot(kind='barh', ax=axes[1], color='orange')
axes[1].set_title('Job Count by Machine')
axes[1].set_xlabel('Number of Jobs')

plt.tight_layout()
plt.show()

## 6. Operator Performance Analysis

In [ ]:
# Operator analysis (exclude unknown operators)
if 'OperatorName' in df_sample.columns:
    operator_data = df_sample[df_sample['OperatorName'] != 'Unknown'].copy()
    
    if len(operator_data) > 0:
        operator_stats = operator_data.groupby('OperatorName').agg({
            'efficiency': ['mean', 'std', 'count'],
            'machine': 'nunique',
            'PartNumber': 'nunique',
            'SetupTime': 'mean'
        }).round(3)
        
        # Flatten column names
        operator_stats.columns = ['_'.join(col).strip() for col in operator_stats.columns]
        operator_stats = operator_stats.rename(columns={
            'efficiency_mean': 'avg_efficiency',
            'efficiency_std': 'efficiency_std',
            'efficiency_count': 'job_count',
            'machine_nunique': 'machines_operated',
            'PartNumber_nunique': 'unique_parts',
            'SetupTime_mean': 'avg_setup_time'
        })
        
        # Filter operators with at least 5 jobs
        experienced_operators = operator_stats[operator_stats['job_count'] >= 5].copy()
        
        if len(experienced_operators) > 0:
            # Calculate versatility score
            experienced_operators['versatility_score'] = experienced_operators['machines_operated'] * experienced_operators['unique_parts']
            
            print("TOP OPERATORS PERFORMANCE (min 5 jobs)")
            print("="*60)
            top_operators = experienced_operators.nlargest(10, 'avg_efficiency')
            print(top_operators[['avg_efficiency', 'job_count', 'machines_operated', 'versatility_score']])
            
            # Visualize operator performance
            fig, axes = plt.subplots(2, 2, figsize=(15, 10))
            
            # Top 10 operators by efficiency
            top_operators['avg_efficiency'].plot(kind='bar', ax=axes[0,0], color='lightcoral')
            axes[0,0].set_title('Top 10 Operators by Efficiency')
            axes[0,0].set_ylabel('Average Efficiency')
            axes[0,0].tick_params(axis='x', rotation=45)
            
            # Efficiency vs consistency scatter
            experienced_operators['consistency_score'] = 1 / (1 + experienced_operators['efficiency_std'])
            axes[0,1].scatter(experienced_operators['avg_efficiency'], experienced_operators['consistency_score'], 
                            s=experienced_operators['job_count']*2, alpha=0.7, color='green')
            axes[0,1].set_xlabel('Average Efficiency')
            axes[0,1].set_ylabel('Consistency Score')
            axes[0,1].set_title('Efficiency vs Consistency (size = job count)')
            
            # Versatility distribution
            axes[1,0].hist(experienced_operators['versatility_score'], bins=20, alpha=0.7, color='purple')
            axes[1,0].set_xlabel('Versatility Score')
            axes[1,0].set_ylabel('Count')
            axes[1,0].set_title('Operator Versatility Distribution')
            
            # Versatility vs efficiency
            axes[1,1].scatter(experienced_operators['versatility_score'], experienced_operators['avg_efficiency'], 
                            alpha=0.7, color='orange')
            axes[1,1].set_xlabel('Versatility Score')
            axes[1,1].set_ylabel('Average Efficiency')
            axes[1,1].set_title('Versatility vs Efficiency')
            
            plt.tight_layout()
            plt.show()
        else:
            print("No operators with sufficient job history (>=5 jobs) found.")
    else:
        print("No operator data available (all operators are 'Unknown').")
else:
    print("No OperatorName column found in the data.")

## 7. Downtime Analysis

In [ ]:
# Downtime category analysis
print("DOWNTIME ANALYSIS")
print("="*50)

# Calculate average downtime by category
downtime_averages = df_sample[available_downtime_cols].mean() / 3600  # Convert to hours
downtime_averages = downtime_averages.sort_values(ascending=False)

print("Average Downtime by Category (Hours):")
for category, avg_time in downtime_averages.items():
    print(f"  {category}: {avg_time:.2f}")

# Visualize downtime categories
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Average downtime by category
downtime_averages.plot(kind='barh', ax=axes[0], color='salmon')
axes[0].set_title('Average Downtime by Category')
axes[0].set_xlabel('Hours')

# Downtime distribution (total)
downtime_total_hours = df_sample['total_downtime'] / 3600
axes[1].hist(downtime_total_hours[downtime_total_hours <= downtime_total_hours.quantile(0.95)], 
            bins=50, alpha=0.7, color='lightblue')
axes[1].set_title('Total Downtime Distribution')
axes[1].set_xlabel('Total Downtime (Hours)')
axes[1].set_ylabel('Frequency')

plt.tight_layout()
plt.show()

In [ ]:
# Downtime correlation with efficiency
if len(available_downtime_cols) > 0:
    # Calculate correlation between downtime categories and efficiency
    downtime_efficiency_corr = df_sample[available_downtime_cols + ['efficiency']].corr()['efficiency'].drop('efficiency')
    downtime_efficiency_corr = downtime_efficiency_corr.sort_values()
    
    print("\nCorrelation between Downtime Categories and Efficiency:")
    for category, corr in downtime_efficiency_corr.items():
        print(f"  {category}: {corr:.3f}")
    
    # Visualize correlation
    plt.figure(figsize=(10, 6))
    colors = ['red' if x < 0 else 'green' for x in downtime_efficiency_corr.values]
    downtime_efficiency_corr.plot(kind='barh', color=colors)
    plt.title('Correlation: Downtime Categories vs Efficiency')
    plt.xlabel('Correlation Coefficient')
    plt.axvline(x=0, color='black', linestyle='-', alpha=0.5)
    plt.tight_layout()
    plt.show()

## 8. Time-based Patterns

In [ ]:
# Time-based analysis
if 'StartTime' in df_sample.columns:
    # Convert to datetime and extract time features
    df_sample['StartTime'] = pd.to_datetime(df_sample['StartTime'])
    df_sample['hour'] = df_sample['StartTime'].dt.hour
    df_sample['day_of_week'] = df_sample['StartTime'].dt.dayofweek
    df_sample['date'] = df_sample['StartTime'].dt.date
    
    # Filter out invalid dates (1969)
    valid_time_data = df_sample[df_sample['StartTime'].dt.year > 1970].copy()
    
    if len(valid_time_data) > 0:
        print("TIME-BASED PATTERNS")
        print("="*50)
        
        # Hourly patterns
        hourly_stats = valid_time_data.groupby('hour').agg({
            'efficiency': 'mean',
            'JobDuration': 'count'
        }).round(3)
        
        # Daily patterns
        day_names = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
        daily_stats = valid_time_data.groupby('day_of_week').agg({
            'efficiency': 'mean',
            'JobDuration': 'count'
        }).round(3)
        daily_stats.index = [day_names[i] for i in daily_stats.index]
        
        # Visualize time patterns
        fig, axes = plt.subplots(2, 2, figsize=(15, 10))
        
        # Hourly efficiency pattern
        hourly_stats['efficiency'].plot(kind='line', marker='o', ax=axes[0,0], color='blue')
        axes[0,0].set_title('Efficiency by Hour of Day')
        axes[0,0].set_xlabel('Hour')
        axes[0,0].set_ylabel('Average Efficiency')
        axes[0,0].grid(True, alpha=0.3)
        
        # Hourly job volume
        hourly_stats['JobDuration'].plot(kind='bar', ax=axes[0,1], color='orange')
        axes[0,1].set_title('Job Volume by Hour of Day')
        axes[0,1].set_xlabel('Hour')
        axes[0,1].set_ylabel('Job Count')
        
        # Daily efficiency pattern
        daily_stats['efficiency'].plot(kind='bar', ax=axes[1,0], color='green')
        axes[1,0].set_title('Efficiency by Day of Week')
        axes[1,0].set_xlabel('Day')
        axes[1,0].set_ylabel('Average Efficiency')
        
        # Daily job volume
        daily_stats['JobDuration'].plot(kind='bar', ax=axes[1,1], color='purple')
        axes[1,1].set_title('Job Volume by Day of Week')
        axes[1,1].set_xlabel('Day')
        axes[1,1].set_ylabel('Job Count')
        
        plt.tight_layout()
        plt.show()
        
        print("\nBest performing hours (by efficiency):")
        best_hours = hourly_stats['efficiency'].nlargest(3)
        for hour, eff in best_hours.items():
            print(f"  {hour}:00 - {eff:.3f}")
            
        print("\nBest performing days (by efficiency):")
        best_days = daily_stats['efficiency'].nlargest(3)
        for day, eff in best_days.items():
            print(f"  {day} - {eff:.3f}")
    else:
        print("No valid time data found (all timestamps are invalid).")
else:
    print("No StartTime column found for time-based analysis.")

## 9. Correlation Analysis

In [ ]:
# Correlation matrix for numerical variables
numerical_cols = df_sample.select_dtypes(include=[np.number]).columns
correlation_matrix = df_sample[numerical_cols].corr()

# Focus on correlations with efficiency
efficiency_corr = correlation_matrix['efficiency'].abs().sort_values(ascending=False)

print("VARIABLES MOST CORRELATED WITH EFFICIENCY")
print("="*50)
for var, corr in efficiency_corr.head(10).items():
    if var != 'efficiency':
        sign = '+' if correlation_matrix.loc['efficiency', var] > 0 else '-'
        print(f"  {var}: {sign}{corr:.3f}")

# Visualize correlation heatmap
plt.figure(figsize=(12, 10))
# Select key variables for heatmap
key_vars = ['efficiency', 'JobDuration', 'RunningTime', 'total_downtime', 'SetupTime', 'PartsProduced']
key_vars = [var for var in key_vars if var in correlation_matrix.columns]

if len(key_vars) > 1:
    key_corr = correlation_matrix.loc[key_vars, key_vars]
    sns.heatmap(key_corr, annot=True, cmap='coolwarm', center=0, 
                square=True, linewidths=0.5, fmt='.3f')
    plt.title('Correlation Heatmap - Key Performance Variables')
    plt.tight_layout()
    plt.show()
else:
    print("Insufficient numerical variables for correlation analysis.")

## 10. Key Insights and Recommendations

In [ ]:
# Generate key insights
print("KEY INSIGHTS FROM DATA EXPLORATION")
print("="*60)

# Data quality insights
print("\n1. DATA QUALITY:")
if quality_report['invalid_dates'] > 0:
    print(f"   - Found {quality_report['invalid_dates']} records with invalid timestamps (1969)")
    print("   - Recommendation: Filter out 1969 timestamps in preprocessing")

if quality_report['negative_durations']:
    print("   - Found negative duration values in some columns")
    print("   - Recommendation: Set negative durations to 0 or implement data validation")

# Performance insights
print("\n2. PERFORMANCE METRICS:")
print(f"   - Average efficiency: {df_sample['efficiency'].mean():.3f}")
print(f"   - Efficiency std dev: {df_sample['efficiency'].std():.3f}")
print(f"   - {(df_sample['efficiency'] > 0.8).sum()} jobs ({(df_sample['efficiency'] > 0.8).mean()*100:.1f}%) have >80% efficiency")

# Machine insights
print("\n3. MACHINE PERFORMANCE:")
if len(machine_stats) > 0:
    best_machine = machine_stats.index[-1]  # Last in sorted order (highest efficiency)
    worst_machine = machine_stats.index[0]  # First in sorted order (lowest efficiency)
    print(f"   - Best performing machine: {best_machine} ({machine_stats.loc[best_machine, 'avg_efficiency']:.3f} efficiency)")
    print(f"   - Worst performing machine: {worst_machine} ({machine_stats.loc[worst_machine, 'avg_efficiency']:.3f} efficiency)")
    efficiency_range = machine_stats['avg_efficiency'].max() - machine_stats['avg_efficiency'].min()
    print(f"   - Efficiency range across machines: {efficiency_range:.3f}")

# Operator insights
if 'experienced_operators' in locals() and len(experienced_operators) > 0:
    print("\n4. OPERATOR PERFORMANCE:")
    print(f"   - {len(experienced_operators)} operators with ≥5 jobs")
    best_operator = experienced_operators['avg_efficiency'].idxmax()
    print(f"   - Best operator: {best_operator} ({experienced_operators.loc[best_operator, 'avg_efficiency']:.3f} efficiency)")
    print(f"   - Average operator versatility: {experienced_operators['versatility_score'].mean():.1f}")

# Time insights
if 'hourly_stats' in locals():
    print("\n5. TIME PATTERNS:")
    best_hour = hourly_stats['efficiency'].idxmax()
    worst_hour = hourly_stats['efficiency'].idxmin()
    print(f"   - Best performing hour: {best_hour}:00 ({hourly_stats.loc[best_hour, 'efficiency']:.3f} efficiency)")
    print(f"   - Worst performing hour: {worst_hour}:00 ({hourly_stats.loc[worst_hour, 'efficiency']:.3f} efficiency)")

# Downtime insights
print("\n6. DOWNTIME ANALYSIS:")
main_downtime = downtime_averages.index[0] if len(downtime_averages) > 0 else "Unknown"
print(f"   - Primary downtime category: {main_downtime} ({downtime_averages.iloc[0]:.2f} hours average)")
print(f"   - Total average downtime per job: {df_sample['total_downtime'].mean()/3600:.2f} hours")

print("\n7. MODELING RECOMMENDATIONS:")
print("   - Use machine and operator as key categorical features")
print("   - Include time-based features (hour, day of week)")
print("   - Consider separate models for different downtime categories")
print("   - Implement data quality filters in preprocessing")
print("   - Focus on efficiency prediction as primary target")
print("   - Consider operator-machine interaction features")

## Summary

This exploration has provided valuable insights into the CNC manufacturing data:

### Data Quality
- Identified data quality issues that need preprocessing
- Found patterns in missing values and outliers

### Performance Patterns
- Efficiency varies significantly across machines and operators
- Clear time-based patterns exist in performance
- Downtime categories have different impacts on overall efficiency

### Modeling Opportunities
- Strong candidates for predictive features identified
- Multiple modeling targets possible (efficiency, downtime, duration)
- Operator-machine combinations show promise for recommendation systems

### Next Steps
1. Implement comprehensive data preprocessing pipeline
2. Engineer additional features based on insights
3. Develop machine learning models for prediction
4. Build operator performance analytics
5. Create recommendation system for optimal assignments